## MV GAUSSIAN - MCMC TEST

In [1]:
using Random,LinearAlgebra, Flux.Optimise, Plots, ForneyLab
using ForneyLab:ParamStr2,renderCVI,iBLR,naturalToBCParams,oneWindowSimulation_MCMC
import StatsBase:percentile,autocor

In [2]:
Random.seed!(1234);

In [3]:
d=2
if d==2
    μ_in = [0.0,0.0]
    s_in = 100*[1 0;0 1.]
    λ_init = [s_in*μ_in;vec(-0.5*s_in)] # Natparams
    msg_in = Message(Multivariate, GaussianWeightedMeanPrecision,xi=s_in*μ_in,w=s_in)
else
    μ_in = 0.0
    s_in = 0.01
    λ_init = [s_in*μ_in,-0.5*s_in] # Natparams
    msg_in = Message(Univariate, GaussianWeightedMeanPrecision,xi=s_in*μ_in,w=s_in)
end



Message: 𝒩(xi=[0.00, 0.00], w=[[1.00e+02, 0.00][0.00, 1.00e+02]])


In [4]:
#f(x) = x
# σ(x) = 1/(1+exp(-x))
# f(x)= [x^2+σ(x)tanh(x)+2*log(abs(x))*x,x]
f(x)= x

if d==2
    μ_out = [f(120.) .+ randn(),f(120.) .+ randn()]
    s_out = 1e12*[1 0;0 1.]
    msg_out = Message(Multivariate, GaussianWeightedMeanPrecision,xi=s_out*μ_out,w=s_out)
else
    μ_out = f(120.) .+ randn()
    s_out = 100
    msg_out = Message(Univariate, GaussianWeightedMeanPrecision,xi=s_out*μ_out,w=s_out) 
end


logp_nc(z) = logPdf(msg_out.dist, f.(z))
println(msg_out.dist)

𝒩(xi=[1.21e+14, 1.19e+14], w=[[1.00e+12, 0.00][0.00, 1.00e+12]])



In [ ]:
opt = ParamStr2(eta=0.5,pareto_k_thr=0.05,max_iterations=1e4,mcmc_window_len=1000,
    rhat_cutoff=4,mcse_cutoff =1e-7,ess_threshold=150.,convergence_algo="MonteCarlo")
#opt= Descent(1e-7)
λ = renderCVI(logp_nc,1000,opt,λ_init,msg_in)
λ_bc = naturalToBCParams(msg_in.dist,λ)
println("μ=$(λ_bc[1]),S=$(λ_bc[2])")

    


In [6]:
(typeof(msg_in.dist) <: Union{ProbabilityDistribution{Multivariate,F},ProbabilityDistribution{Univariate,F}} where F<:Gaussian)

true

In [7]:
# opts = ParamStr2(eta=0.1,pareto_k_thr=0.05,max_iterations=1e5,mcmc_window_len=1000,
#     rhat_cutoff=4,mcse_cutoff =1e-7,ess_threshold=150.,convergence_algo="MonteCarlo")
# opts.current_stepsize=opts.eta
# J=11
# Window=1000
# if d==2
#     η=[0.0, 0.01]
# λ_initials=[[randn(),randn()],1e12*[1 0;0 1.],1e1*[1 0;0 1.]]
# else
#     η=[0.0, 0.01]
#     λ_initials=[0.0, 0.01]
# end

# last_params,opt_matrix,stats_dict=oneWindowSimulation_MCMC(J,Window,opts,η,λ_initials,logp_nc,true,msg_in);
# stats_dict